In [4]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image 

In [5]:
## loading training data
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/face-mask-dataset/data',
    validation_split=0.2,
    subset= "training",
    seed=42,
    image_size= (200,200),
    batch_size=50
)
    

In [12]:
## loading testing data
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/face-mask-dataset/data',
    validation_split=0.2,
    subset= "validation",
    seed=42,
    image_size= (200,200),
    batch_size=50
)
    

In [7]:
class_names = training_ds.class_names
class_names

In [8]:
plt.figure(figsize=(10, 10))
for images, labels in training_ds.take(1):
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.grid(True)

In [9]:
mymodel = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(2, activation= 'softmax')
    
    
])

In [10]:
mymodel.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
retVal= mymodel.fit(training_ds, validation_data= testing_ds, epochs = 20)

In [25]:
plt.plot(retVal.history['loss'],label="training loss")
plt.plot(retVal.history['accuracy'],label="training accuracy")
plt.legend()

In [26]:
plt.figure(figsize=(20, 20))
for images, labels in testing_ds.take(1):
    predictions = mymodel.predict(images)
    predlabel = []
    
    for val in predictions:
        predlabel.append(class_names[np.argmax(val)])
    
    for i in range(40):
        ax = plt.subplot(10, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title('Predicted label:'+ predlabel[i])
        plt.axis('off') 

In [29]:
mymodel.save('maskDetector.model')